<a href="https://colab.research.google.com/github/rpasquini/defidata/blob/main/AAVE_Unhealthy_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unhealthy positions Monitor

In [3]:
import requests
from pprint import pprint
from datetime import datetime
import pandas as pd

In [4]:
#(See https://thegraph.com/hosted-service/subgraph/aave/protocol-v2?version=current)
subgraph_http_address='https://api.thegraph.com/subgraphs/name/aave/protocol-v2'

# function to use requests.post to make an API call to the subgraph url
def run_query(query):

    # endpoint where you are making the request
    request = requests.post(subgraph_http_address,
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

In [70]:
#users(first:1000, skip: """+str(1000*count)+""", orderBy: id, orderDirection: desc, where: {borrowedReservesCount_gt: 0}) {
count=0
first=100
query="""{
users(first:"""+str(first)+""", skip: """+str(1000*count)+""", orderBy: id, orderDirection: desc, where: {borrowedReservesCount_gt: 0}){
          id
          borrowedReservesCount
          collateralReserve:reserves(where: {currentATokenBalance_gt: 0}) {
            currentATokenBalance
            reserve{
              usageAsCollateralEnabled
              reserveLiquidationThreshold
              reserveLiquidationBonus
              borrowingEnabled
              utilizationRate
              symbol
              underlyingAsset
              decimals
              price {
                priceInEth
              }

            }
          }
          borrowReserve: reserves(where: {currentTotalDebt_gt: 0}) {
            currentTotalDebt
            reserve{
              usageAsCollateralEnabled
              reserveLiquidationThreshold
              borrowingEnabled
              utilizationRate
              symbol
              underlyingAsset
              decimals
              price {
                priceInEth
              }

            }
          }
      }
      }
"""
result = run_query(query)
result

{'data': {'users': [{'id': '0xffff8941130157a0153fb5be2618b257f28d3b55',
    'borrowedReservesCount': 1,
    'collateralReserve': [{'currentATokenBalance': '385716688415711350',
      'reserve': {'usageAsCollateralEnabled': True,
       'reserveLiquidationThreshold': '8600',
       'reserveLiquidationBonus': '10500',
       'borrowingEnabled': True,
       'utilizationRate': '0.53245163',
       'symbol': 'WETH',
       'underlyingAsset': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
       'decimals': 18,
       'price': {'priceInEth': '1000000000000000000'}}}],
    'borrowReserve': [{'currentTotalDebt': '353684533943004137474',
      'reserve': {'usageAsCollateralEnabled': True,
       'reserveLiquidationThreshold': '8700',
       'borrowingEnabled': True,
       'utilizationRate': '0.31296075',
       'symbol': 'DAI',
       'underlyingAsset': '0x6b175474e89094c44da98b954eedeac495271d0f',
       'decimals': 18,
       'price': {'priceInEth': '841484886277113'}}}]},
   {'id': '0xfffc

In [7]:
userdata0=result['data']['users'][0]
userdata0

{'id': '0xffff8941130157a0153fb5be2618b257f28d3b55',
 'borrowedReservesCount': 1,
 'collateralReserve': [{'currentATokenBalance': '385716688415711350',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8600',
    'reserveLiquidationBonus': '10500',
    'borrowingEnabled': True,
    'utilizationRate': '0.53359029',
    'symbol': 'WETH',
    'underlyingAsset': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
    'price': {'priceInEth': '1000000000000000000'},
    'decimals': 18}}],
 'borrowReserve': [{'currentTotalDebt': '353684533943004137474',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8700',
    'borrowingEnabled': True,
    'utilizationRate': '0.31309154',
    'symbol': 'DAI',
    'underlyingAsset': '0x6b175474e89094c44da98b954eedeac495271d0f',
    'price': {'priceInEth': '841484886277113'},
    'decimals': 18}}]}

In [28]:
385716688415711350*(10**-18)

0.3857166884157114

In [34]:
def test(userdata):

    """Computes total collateral times threshold -across all reserves/currencies - (in ETH)
    """
    total_collateral=0
    total_collateral_threshold=0

    for collateral_reserve in userdata["collateralReserve"]:
        decimals=int(collateral_reserve["reserve"]["decimals"])
        print('decimals',decimals)
        price_in_eth = int(collateral_reserve["reserve"]["price"]["priceInEth"])/ (10 ** decimals)
        print('price in eth',price_in_eth)
        current_a_token_balance = int(collateral_reserve["currentATokenBalance"])/ (10 ** decimals)
        print('current_a_token_balance',current_a_token_balance)
        total_collateral +=price_in_eth * current_a_token_balance
        print('total_collateral',total_collateral)
        liquidation_threshold=(int(collateral_reserve['reserve']['reserveLiquidationThreshold']) / 10000)
        print('liquidation_threshold',liquidation_threshold)

        #sum of collateral times thresholds 
        total_collateral_threshold += price_in_eth * current_a_token_balance * liquidation_threshold
        print('total_collateral_threshold',total_collateral_threshold)
            

    return total_collateral_threshold

test(userdata0)

decimals 18
price in eth 1.0
current_a_token_balance 0.38571668841571133
total_collateral 0.38571668841571133
liquidation_threshold 0.86
total_collateral_threshold 0.33171635203751176


0.33171635203751176

In [59]:
def compute_total_borrowed(userdata):

  """Computes total borrowed  -across all reserves/currencies - (in ETH)
  """
  total_borrowed = 0
  # Three additional variables to detect the largest borrow across all currencies.
  #max_borrowed_symbol = None
  #max_borrowed_principal = 0
  #max_borrowed_price_in_eth = 0

  for borrow_reserve in userdata["borrowReserve"]:
            decimals=int(borrow_reserve["reserve"]["decimals"])
            price_in_eth = int(borrow_reserve["reserve"]["price"]["priceInEth"]) / (10 ** 18 )
            principal_borrowed = int(borrow_reserve["currentTotalDebt"]) / (10 **decimals )
            total_borrowed += price_in_eth * principal_borrowed
            
            #if principal_borrowed > max_borrowed_principal:
                #max_borrowed_symbol = borrow_reserve["reserve"]["symbol"]
                #max_borrowed_principal = principal_borrowed
                #max_borrowed_price_in_eth = price_in_eth
  
  return  total_borrowed

def compute_total_collateral_times_thresholds(userdata, print=False):

    """Computes total collateral times threshold -across all reserves/currencies - (in ETH)
    """

    """Computes total collateral times threshold -across all reserves/currencies - (in ETH)
    """
    total_collateral=0
    total_collateral_threshold=0

    for collateral_reserve in userdata["collateralReserve"]:
        decimals=int(collateral_reserve["reserve"]["decimals"])

        price_in_eth = int(collateral_reserve["reserve"]["price"]["priceInEth"])/ (10 ** 18)

        current_a_token_balance = int(collateral_reserve["currentATokenBalance"])/ (10 ** decimals)

        total_collateral +=price_in_eth * current_a_token_balance

        liquidation_threshold=(int(collateral_reserve['reserve']['reserveLiquidationThreshold']) / 10000)


        #sum of collateral times thresholds 
        total_collateral_threshold += price_in_eth * current_a_token_balance * liquidation_threshold


        if print: 
          print('decimals',decimals)
          print('price in eth',price_in_eth)
          print('current_a_token_balance',current_a_token_balance)
          print('total_collateral',total_collateral)
          print('liquidation_threshold',liquidation_threshold)
          print('total_collateral_threshold',total_collateral_threshold)

    return total_collateral_threshold

def compute_health_factor(userdata):

    """Computes health factor
    """
    total_borrowed=compute_total_borrowed(userdata)
    total_collateral_threshold=compute_total_collateral_times_thresholds(userdata)
    health_factor = total_collateral_threshold / total_borrowed
    
    return health_factor


def parse_users(queryresult):
  health_factors=[]
  for userdata in queryresult['data']['users']:
    health_factor=compute_health_factor(userdata)
    health_factors.append(health_factor)
  return health_factors


  #for borrow_reserve in user["borrowReserve"]:
    #borrow_reserve["reserve"]["price"]["priceInEth"]

In [60]:
compute_total_borrowed(userdata0)

0.29762018982300253

In [61]:
compute_total_collateral_times_thresholds(userdata0)

0.33171635203751176

In [62]:
compute_health_factor(userdata0)

1.114562665371548

In [73]:
parse_users(result)

[1.114562665371548,
 1.4270831320454729,
 0.5152897903881213,
 3.7502622037361,
 0.975712874237369,
 3.27830520078925,
 2.056611013952261,
 0.8870117038521624,
 0.22305881374036168,
 0.9986993109665995,
 1.0849235174420002,
 0.5322070223112975,
 8.747071906493892,
 0.7317846556780339,
 1.3910704676104153,
 4.878569249756143,
 1.1478981743370182,
 0.7319254105646756,
 0.3797097386267468,
 1.0773848635998018,
 2.258254825392238,
 0.0597943117284452,
 0.8411752725745409,
 2.0023766339392637,
 3.2523762474579283,
 1.164260714818336,
 2.9271386227121354,
 1.384315163696773,
 1.5736164596087658,
 1.3511645680605973,
 20037.673181970393,
 1.9167175110848291,
 1.7562831736272797,
 2.076296486104553,
 4.390703543364661,
 11.978534705441977,
 1.1254070292469367,
 0.5135330646758275,
 0.8771943046840116,
 1.3514028729049565,
 0.9500047944935858,
 0.5420627079096548,
 0.8978330975334109,
 0.5274123644526371,
 1.0570222804238265,
 2.433623346141777,
 3.644094900243717,
 1.3296028674930358,
 1.30095

In [ ]:
compute_total_borrowed(datauser0)
compute_total_collateral_times_thresholds(datauser0)

In [66]:
datauser1=result['data']['users'][2]
datauser1

{'id': '0xfffb40347816873ca038756cc87bc610144166b2',
 'borrowedReservesCount': 1,
 'collateralReserve': [{'currentATokenBalance': '1003420809121393',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8300',
    'reserveLiquidationBonus': '10700',
    'borrowingEnabled': False,
    'utilizationRate': '-0.00003649',
    'symbol': 'STETH',
    'underlyingAsset': '0xae7ab96520de3a18e5e111b5eaab095312d7fe84',
    'price': {'priceInEth': '989207430000000000'},
    'decimals': 18}}],
 'borrowReserve': [{'currentTotalDebt': '1900000',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8750',
    'borrowingEnabled': True,
    'utilizationRate': '0.43185064',
    'symbol': 'USDC',
    'underlyingAsset': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    'price': {'priceInEth': '841479280000000'},
    'decimals': 6}}]}

In [68]:
print(compute_total_borrowed(datauser1))
print(compute_total_collateral_times_thresholds(datauser1))
print(compute_health_factor(datauser1))

0.0015988106319999998
0.0008238507954335797
0.5152897903881213


In [58]:
1/(841479280000000*10**(-18))

1188.38339073542

In [ ]:
def parse_users(payload):
    loans = []

    for user in payload["users"]:
        total_borrowed = 0
        total_collateral = 0
        total_collateral_threshold = 0
        max_borrowed_symbol = None
        max_borrowed_principal = 0
        max_borrowed_price_in_eth = 0
        max_collateral_symbol = None
        max_collateral_bonus = 0
        max_collateral_price_in_eth = 0

        for borrow_reserve in user["borrowReserve"]:
            price_in_eth = borrow_reserve["reserve"]["price"]["priceInEth"]
            principal_borrowed = borrow_reserve["currentTotalDebt"]
            total_borrowed += (
                price_in_eth * principal_borrowed
                / 10 ** borrow_reserve["reserve"]["decimals"]
            )
            if principal_borrowed > max_borrowed_principal:
                max_borrowed_symbol = borrow_reserve["reserve"]["symbol"]
                max_borrowed_principal = principal_borrowed
                max_borrowed_price_in_eth = price_in_eth

        for collateral_reserve in user["collateralReserve"]:
            price_in_eth = collateral_reserve["reserve"]["price"]["priceInEth"]
            principal_a_token_balance = collateral_reserve["currentATokenBalance"]
            total_collateral += (
                price_in_eth * principal_a_token_balance
                / 10 ** collateral_reserve["reserve"]["decimals"]
            )
            total_collateral_threshold += (price_in_eth * principal_a_token_balance * (collateral_reserve['reserve']['reserveLiquidationThreshold'] / 10000)) / 10**collateral_reserve['reserve']['decimals']
            if collateral_reserve['reserve']['reserveLiquidationBonus'] > max_collateral_bonus:
              max_collateral_symbol = collateral_reserve['reserve']['symbol']
              max_collateral_bonus = collateral_reserve['reserve']['reserveLiquidationBonus']
              max_collateral_price_in_eth = price_in_eth

    health_factor = total_collateral_threshold / total_borrowed

    if health_factor <= health_factor_max:
        loans.append({
            "user_id": user['id'],
            "healthFactor": health_factor,
            "max_collateralSymbol": max_collateral_symbol,
            "max_borrowedSymbol": max_borrowed_symbol,
            "max_borrowedPrincipal": max_borrowed_principal,
            "max_borrowedPriceInEth": max_borrowed_price_in_eth,
            "max_collateralBonus": max_collateral_bonus / 10000,
            "max_collateralPriceInEth": max

{'errors': [{'locations': [{'line': 2, 'column': 1}],
   'message': 'Unexpected `users[Name]`\nExpected `{`, `query`, `mutation`, `subscription` or `fragment`'}]}

In [ ]:
    for i, borrow_reserve in enumerate(user['borrowReserve']):
        price_in_eth = borrow_reserve['reserve']['price']['priceInEth']
        principal_borrowed = borrow_reserve['currentTotalDebt']
        total_borrowed += (price_in_eth * principal_borrowed) / 10**borrow_reserve['reserve']['decimals']
        if principal_borrowed > max_borrowed_principal:
            max_borrowed_symbol = borrow_reserve['reserve']['symbol']
            max_borrowed_principal = principal_borrowed
            max_borrowed_price_in_eth = price_in_eth

    for i, collateral_reserve in enumerate(user['collateralReserve']):
        price_in_eth = collateral_reserve['reserve']['price']['priceInEth']
        principal_a_token_balance = collateral_reserve['currentATokenBalance']
        total_collateral += (price_in_eth * principal_a_token_balance) / 10**collateral_reserve['reserve']['decimals']
        total_collateral_threshold += (price_in_eth * principal_a_token_balance * (collateral_reserve['reserve']['reserveLiquidationThreshold'] / 10000)) / 10**collateral_reserve['reserve']['decimals']
        if collateral_reserve['reserve']['reserveLiquidationBonus'] > max_collateral_bonus:
            max_collateral_symbol = collateral_reserve['reserve']['symbol']
            max_collateral_bonus = collateral_reserve['reserve']['reserveLiquidationBonus']
            max_collateral_price_in_eth = price_in_eth

    health_factor = total_collateral_threshold / total_borrowed

    if health_factor <= health_factor_max:
        loans.append({
            "user_id": user['id'],
            "healthFactor": health_factor,
            "max_collateralSymbol": max_collateral_symbol,
            "max_borrowedSymbol": max_borrowed_symbol,
            "max_borrowedPrincipal": max_borrowed_principal,
            "max_borrowedPriceInEth": max_borrowed_price_in_eth,
            "max_collateralBonus": max_collateral_bonus / 10000,
            "max_collateralPriceInEth": max
